In [1]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'velox.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

import pandas as pd
from velox_app.models import Horse, Measure

In [2]:
measures = Measure.objects.all()
len(measures)

11003

In [3]:
measures_dict = [m.__dict__ for m in measures]
df_measures = pd.DataFrame(measures_dict)
df_measures.drop(columns=['_state'], inplace=True)

In [46]:
from decimal import Decimal
df_measuresx = df_measures[~df_measures['prob_cardio_bio_model'].isna()]
df_measuresx = df_measuresx['prob_cardio_bio_model'].apply(lambda x: Decimal(str(x)))
res = pd.qcut(df_measuresx, q=6,  retbins=True)

In [51]:
pd.qcut(df_measuresx.unique(), q=6, retbins=True)

([(0.449, 0.831], (0.449, 0.831], (0.449, 0.831], (0.449, 0.831], (0.449, 0.831], ..., (0.26, 0.449], (0.26, 0.449], (0.26, 0.449], (0.26, 0.449], (0.26, 0.449]]
 Length: 3543
 Categories (6, interval[float64, right]): [(0.0687, 0.0967] < (0.0967, 0.129] < (0.129, 0.159] < (0.159, 0.26] < (0.26, 0.449] < (0.449, 0.831]],
 array([0.06973092, 0.09674302, 0.12873187, 0.15856479, 0.26005678,
        0.44886125, 0.83053434]))

In [40]:
res[0].value_counts()

D     833
C     832
B     832
B+    832
A     832
A+    832
Name: prob_cardio_bio_model, dtype: int64

In [59]:
# res[1]
bins_values = res[1]
bins = {
    'D': [0.0, bins_values[1]],
    'C': [bins_values[1], bins_values[2]],
    'B': [bins_values[2], bins_values[3]],
    'B+': [bins_values[3], bins_values[4]],
    'A': [bins_values[4], bins_values[5]],
    'A+': [bins_values[5], 1.0],
}
bins_rev = {
    (0.0, bins_values[1]): 'D',
    (bins_values[1], bins_values[2]): 'C',
    (bins_values[2], bins_values[3]): 'B',
    (bins_values[3], bins_values[4]): 'B+',
    (bins_values[4], bins_values[5]): 'A',
    (bins_values[5], 1.0): 'A+',
}
from pprint import pprint

pprint(bins)

{'A': [0.2505566478, 0.4478050768],
 'A+': [0.4478050768, 1.0],
 'B': [0.1255440116, 0.1556581855],
 'B+': [0.1556581855, 0.2505566478],
 'C': [0.094587855, 0.1255440116],
 'D': [0.0, 0.094587855]}


In [70]:
categories = res[0].values
# categories.categories.values.to_tuples()
res[0].value_counts()

(0.0687, 0.0946]    833
(0.0946, 0.126]     832
(0.126, 0.156]      832
(0.156, 0.251]      832
(0.251, 0.448]      832
(0.448, 0.831]      832
Name: prob_cardio_bio_model, dtype: int64

In [58]:
prob = 0.1
for k,v in bins_rev.items():
    bin_min = k[0]
    bin_max = k[1]
    if bin_max>= prob > bin_min:
        print(v)

C
